In [ ]:
# Welcome to your new notebook
# Type here in the cell editor to add code!


In [1]:
df = spark.read.format("csv").option("header","true").load("Files/Bronzebronze_customer_master/customer_master.csv")
# df now is a Spark DataFrame containing CSV data from "Files/Bronzebronze_customer_master/customer_master.csv".
display(df)

StatementMeta(, 4051b240-90cc-4bd6-8cef-d6b936f1377c, 3, Finished, Available, Finished, False)

SynapseWidget(Synapse.DataFrame, efe9cfe6-4e4e-42ca-8274-8e849730e47a)

In [3]:
from pyspark.sql.functions import col, trim, upper, initcap

df_cust_silver = (
    df
    .withColumn("CustomerID", trim(upper(col("CustomerID"))))
    .withColumn("CustomerName", initcap(trim(col("CustomerName"))))
    .withColumn("Region", initcap(trim(col("Region"))))
    .withColumn("RiskCategory", initcap(trim(col("RiskCategory"))))
)


StatementMeta(, 4051b240-90cc-4bd6-8cef-d6b936f1377c, 5, Finished, Available, Finished, False)

In [4]:
from pyspark.sql.functions import when

df_cust_silver = df_cust_silver.withColumn(
    "RiskCategory",
    when(col("RiskCategory").isNull(), "Unknown")
    .otherwise(col("RiskCategory"))
)


StatementMeta(, 4051b240-90cc-4bd6-8cef-d6b936f1377c, 6, Finished, Available, Finished, False)

In [5]:
from pyspark.sql.types import IntegerType, DecimalType

df_cust_silver = (
    df_cust_silver
    .withColumn("CreditLimit", col("CreditLimit").cast(DecimalType(18,2)))
    .withColumn("CreditDays", col("CreditDays").cast(IntegerType()))
)


StatementMeta(, 4051b240-90cc-4bd6-8cef-d6b936f1377c, 7, Finished, Available, Finished, False)

In [6]:
df_cust_silver = df_cust_silver.dropDuplicates(["CustomerID"])

StatementMeta(, 4051b240-90cc-4bd6-8cef-d6b936f1377c, 8, Finished, Available, Finished, False)

In [7]:
from pyspark.sql.functions import monotonically_increasing_id

df_cust_silver = df_cust_silver.withColumn(
    "CustomerKey",
    monotonically_increasing_id()
)

StatementMeta(, 4051b240-90cc-4bd6-8cef-d6b936f1377c, 9, Finished, Available, Finished, False)

In [8]:
from pyspark.sql.window import Window
from pyspark.sql.functions import row_number

window_spec = Window.orderBy("CustomerID")

df_cust_silver = df_cust_silver.withColumn(
    "CustomerKey",
    row_number().over(window_spec)
)


StatementMeta(, 4051b240-90cc-4bd6-8cef-d6b936f1377c, 10, Finished, Available, Finished, False)

In [9]:
display(df_cust_silver)

StatementMeta(, 4051b240-90cc-4bd6-8cef-d6b936f1377c, 11, Finished, Available, Finished, False)

SynapseWidget(Synapse.DataFrame, 92cf7dee-d48d-42b1-9650-81e5a8efc3d3)

In [10]:
from pyspark.sql.functions import current_date, lit

df_cust_silver = (
    df_cust_silver
    .withColumn("EffectiveFrom", current_date())
    .withColumn("EffectiveTo", lit(None).cast("date"))
    .withColumn("IsCurrent", lit(1))
)

StatementMeta(, 4051b240-90cc-4bd6-8cef-d6b936f1377c, 12, Finished, Available, Finished, False)

In [12]:
df_cust_silver.printSchema()
display(df_cust_silver)


StatementMeta(, 4051b240-90cc-4bd6-8cef-d6b936f1377c, 14, Finished, Available, Finished, False)

root
 |-- CustomerID: string (nullable = true)
 |-- CustomerName: string (nullable = true)
 |-- Region: string (nullable = true)
 |-- CreditLimit: decimal(18,2) (nullable = true)
 |-- CreditDays: integer (nullable = true)
 |-- RiskCategory: string (nullable = true)
 |-- CustomerKey: integer (nullable = false)
 |-- EffectiveFrom: date (nullable = false)
 |-- EffectiveTo: date (nullable = true)
 |-- IsCurrent: integer (nullable = false)



SynapseWidget(Synapse.DataFrame, 67c66a29-1d2c-4da5-a339-bf4eddf3734c)

In [14]:
silver_path = "abfss://Fabric_Test@onelake.dfs.fabric.microsoft.com/Test_Lakehouse.Lakehouse/Files/Silver Layer Files"
df_cust_silver.write.mode("overwrite") \
    .parquet(f"{silver_path}/silver_customers")

StatementMeta(, 4051b240-90cc-4bd6-8cef-d6b936f1377c, 16, Finished, Available, Finished, False)

In [16]:
df_check = spark.read.parquet(f"{silver_path}/silver_customers")
display(df_check)


StatementMeta(, 4051b240-90cc-4bd6-8cef-d6b936f1377c, 18, Finished, Available, Finished, False)

SynapseWidget(Synapse.DataFrame, c34ab057-e659-458f-9aa1-46d476ebaaba)